### Master Thesis Project: Finding optimal EV charging infrastructure on DFDS routes
_________________________________________________________________________________________

#### Students: Jan Gaydoul & Emanuela Zucchetto

#### Code Part 2: Using Graph Theory and the NetworkX package to find ideal locations for DFDS eTrucks charging stations

In [5]:
# Importing needed libraries

import pandas as pd
import os
pd.set_option('display.max_columns', None)

import networkx as nx
import matplotlib.pyplot as plt


In [6]:
#  Preparing dataset

# set working directory to where the CSV files are located
os.chdir('/Users/jangaydoul/Desktop/Copenhagen Business School/4. Semester :: Thesis/03_Data/csvs')

# initialize an empty dataframe to hold the concatenated data
df_graph = pd.DataFrame()

# loop through each CSV file and append its data to the dataframe
for file in os.listdir():
    if file.endswith('.csv'):
        temp_df = pd.read_csv(file)
        df_graph = pd.concat([df_graph, temp_df])

# export the concatenated data to a new CSV file
df_graph.to_csv('df_graph.csv', index=False)

In [9]:
# len(df_graph)
df_graph['route_distance'].describe()

count    162112.000000
mean        822.890598
std         683.135621
min          -1.000000
25%         274.896750
50%         612.693000
75%        1221.561500
max       12065.484000
Name: route_distance, dtype: float64

In [ ]:
### 1) Building the Graph

In [ ]:
### 2) Visualizations

In [ ]:
### 3) Finding the ideal locations for charging stations 